# A3Data - Residência Porto Digital

## Processamento de Dados em Larga Escala - Aula 04

---

### Setup

In [1]:
%%bash
# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

In [3]:
%%bash
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-gols.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-full.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-estatisticas-full.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-cartoes.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/Legenda.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  270k  100  270k    0     0  2114k      0 --:--:-- --:--:-- --:--:-- 2098k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  825k  100  825k    0     0  5094k      0 --:--:-- --:--:-- --:--:-- 5094k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  600k  100  600k    0     0  5718k      0 --:--:-- --:--:-- --:--:-- 5773k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  855k  100  855k    0     0  4621k      0 --:--:-- --:--:-- --:--:-- 4621k
  % Total    % Received % Xferd  Average Speed   Tim

In [4]:
parametros = {
    'header': True,
    'inferSchema': True,
    'sep': ','
}

schema_full = """ID int, rodada int, data string, hora string, dia string, mandante string, visitante string, formacao_mandante string, 
  formacao_visitante string, tecnico_mandante string, tecnico_visitante string, vencedor string, arena string, mandante_placar string,
  visitante_placar string, mandante_estado string, visitante_estado string, estado_vencedor string"""

cartoes = spark.read.csv("campeonato-brasileiro-cartoes.csv", **parametros)
gols = spark.read.csv("campeonato-brasileiro-gols.csv", **parametros)
estats = spark.read.csv("campeonato-brasileiro-estatisticas-full.csv", **parametros)
full = spark.read.csv("campeonato-brasileiro-full.csv", **parametros, schema=schema_full)

cartoes.createOrReplaceTempView('cartoes')
gols.createOrReplaceTempView('gols')
estats.createOrReplaceTempView('estats')
full.createOrReplaceTempView('full')

# DEVER DE CASA

1. Qual foi o time que mais venceu partidas desde 2014?
2. Qual foi o estado que teve mais derrotas entre 2008 e 2016?
3. Qual foi a média de cartões vermelhos que o Palmeiras tomou nas partidas em que jogou fora de casa, marcou pelo menos 1 gol no primeiro tempo?

In [14]:
q("""

  SELECT
    vencedor, COUNT(1) as qtd_vitorias
  FROM full
  WHERE data >= '2014-01-01'
    AND vencedor <> '-'
  GROUP BY vencedor
  ORDER BY qtd_vitorias DESC
  LIMIT 10

""")

+-------------+------------+
|vencedor     |qtd_vitorias|
+-------------+------------+
|Flamengo     |155         |
|Palmeiras    |148         |
|Atletico-MG  |145         |
|Corinthians  |132         |
|Gremio       |132         |
|Santos       |131         |
|Sao Paulo    |127         |
|Athletico-PR |122         |
|Internacional|116         |
|Fluminense   |112         |
+-------------+------------+



In [32]:
# Qual foi o estado que teve mais derrotas entre 2008 e 2016?
q("""

  WITH tabela_derrotas AS (
    SELECT 
      ID, data,
      mandante_estado, visitante_estado, estado_vencedor,
      CASE
        WHEN mandante_estado = estado_vencedor AND visitante_estado <> estado_vencedor THEN visitante_estado
        WHEN mandante_estado <> estado_vencedor AND visitante_estado = estado_vencedor THEN mandante_estado
        ELSE '-'
      END AS estado_derrota
    FROM full
    WHERE data >= '2008-01-01' AND data <= '2016-12-31'
  )
  SELECT 
    estado_derrota, COUNT(1) as qtd_derrotas
  FROM tabela_derrotas
  WHERE estado_derrota <> '-'
  GROUP BY estado_derrota
  ORDER BY COUNT(1) DESC

""")

+--------------+------------+
|estado_derrota|qtd_derrotas|
+--------------+------------+
|SP            |448         |
|RJ            |362         |
|MG            |262         |
|SC            |255         |
|PR            |227         |
|RS            |196         |
|PE            |191         |
|BA            |152         |
|GO            |147         |
|CE            |30          |
+--------------+------------+



In [56]:
# Qual foi a quantidade de cartões vermelhos que o Palmeiras tomou nas 
# partidas em que jogou fora de casa, marcou pelo menos 1 gol no primeiro tempo?
q("""
  WITH cartoes_vermelhos AS (
    SELECT DISTINCT
      c.*
    FROM cartoes c
    INNER JOIN gols g
      ON c.partida_id = g.partida_id
    INNER JOIN full f
      ON c.partida_id = f.ID
    WHERE c.cartao = 'Vermelho'
      AND c.clube = 'Palmeiras'
      AND g.clube = 'Palmeiras'
      AND (
        g.minuto < 46 
        OR g.minuto LIKE '45%'
      )
      AND f.visitante = 'Palmeiras'
  )
  SELECT 
    COUNT(1) as qtd_vermelhos
  FROM cartoes_vermelhos

""")


+-------------+
|qtd_vermelhos|
+-------------+
|11           |
+-------------+



In [59]:
# Qual foi a média de cartões vermelhos por partida que o Palmeiras tomou nas 
# partidas em que jogou fora de casa, marcou pelo menos 1 gol no primeiro tempo?
q("""
  WITH cartoes_vermelhos AS (
    SELECT DISTINCT
      c.*
    FROM cartoes c
    INNER JOIN gols g
      ON c.partida_id = g.partida_id
    INNER JOIN full f
      ON c.partida_id = f.ID
    WHERE c.cartao = 'Vermelho'
      AND c.clube = 'Palmeiras'
      AND g.clube = 'Palmeiras'
      AND (
        g.minuto < 46 
        OR g.minuto LIKE '45%'
      )
      AND f.visitante = 'Palmeiras'
  )
  SELECT 
    SUM(qtd_vermelhos) / COUNT(1) AS media_cartoes_vermelhos
  FROM (
    SELECT 
      partida_id, COUNT(1) as qtd_vermelhos
    FROM cartoes_vermelhos
    GROUP BY partida_id
  )

""")

+-----------------------+
|media_cartoes_vermelhos|
+-----------------------+
|1.375                  |
+-----------------------+



# Python

Hello World em Java:

```java
class Simple {  
   public static void main(String args[]) {  
      System.out.println("Hello World from Java");     
   }  
}  
```

In [60]:
print("Hello World from Python")

Hello World from Python


In [61]:
a = 1

In [62]:
a

1

In [63]:
type(a)

int

In [64]:
a = 1.0

In [65]:
a

1.0

In [66]:
type(a)

float

Node.js para desenvolvimento

.Net e GoLang pra sistemas e APIs

Python - R, Julia para dados

Python - pra tudo S2

In [67]:
nome = 'Neylson'
sobrenome = 'Crepalde'

In [68]:
type(nome)

str

In [70]:
a = 1
b = 2.0

In [71]:
a + b

3.0

In [73]:
nome + ' ' + sobrenome

'Neylson Crepalde'

In [74]:
sim = True
nao = False

In [75]:
type(sim)

bool

In [94]:
idade = 30
nome = 'Ney'

In [101]:
nome == 'ney'

False

In [85]:
# Se idade for maior que 30, véi. Se não, jovem.
if idade >= 30:
  print("Véi")
  print(idade)
  print("Saída da primeira opcao")
else:
  print("Xóvem")
  print(idade)
  print("Saída da segunda opcao")

Véi
30
Saída da primeira opcao


In [109]:
idade = 29
nome = 'Carlos'

In [110]:
if idade > 30 and nome != 'Ney':
  print("Véi chato")
elif idade > 30 and nome == 'Ney':
  print('Véi legal')
else:
  print('Clausula de senao')

Clausula de senao


Loops - Laços

In [116]:
x = 0
while x < 10:
  print(x)
  x += 1

0
1
2
3
4
5
6
7
8
9


In [112]:
x

10

In [117]:
x = 0
while True:
  print(x)
  x += 1
  if x > 10:
    break

0
1
2
3
4
5
6
7
8
9
10


In [119]:
# Laço for
nomes = ['Ney', 'Rarissa', 'Pedro', 'Luiz', 'Vitória', 'Rayane', 'Levi']

for nome in nomes:
  print(nome + ' é tão legal...')

Ney é tão legal...
Rarissa é tão legal...
Pedro é tão legal...
Luiz é tão legal...
Vitória é tão legal...
Rayane é tão legal...
Levi é tão legal...


In [120]:
for nome in sorted(nomes):
  print(nome + ' é meu coleguinha...')

Levi é meu coleguinha...
Luiz é meu coleguinha...
Ney é meu coleguinha...
Pedro é meu coleguinha...
Rarissa é meu coleguinha...
Rayane é meu coleguinha...
Vitória é meu coleguinha...


In [121]:
for numero in range(10):
  print(numero)

0
1
2
3
4
5
6
7
8
9


In [126]:
nomes[2]

'Pedro'

In [133]:
2%3 # % é o resto da divisão

2

In [136]:
# Printa par ou ímpar
for numero in range(10):
  if numero%2 == 0:
    print(str(numero) + "\t Par!")
  else:
    print(str(numero) + "\t Ímpar!")

0	 Par!
1	 Ímpar!
2	 Par!
3	 Ímpar!
4	 Par!
5	 Ímpar!
6	 Par!
7	 Ímpar!
8	 Par!
9	 Ímpar!


In [137]:
type(range(10))

range

In [141]:
list(range(10)) + [1]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1]

In [142]:
set( list(range(10)) + [1] )

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [144]:
# tuplas
tupla = (0, 1)

In [145]:
print(tupla)
type(tupla)

(0, 1)


tuple

In [146]:
lista = [0, 1, 2]
tupla = (0, 1, 2)

In [148]:
lista[0] = 8
print(lista)

[8, 1, 2]


In [150]:
tupla[0] = 8 # Não consigo modificar elementos de uma tupla

TypeError: ignored

# Semana que vem
Semana que vem vamos trabalhar com orientação a objetos e vamos fazer o famoso projeto do Cachorro!